In [ ]:
import torch
import torch.nn as nn
from torchvision import models, transforms
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt


# Custom Guided Backpropagation ReLU
class GuidedBackpropReLU(nn.Module):
    def forward(self, input):
        return torch.relu(input)

    def backward(self, grad_output):
        grad_input = torch.clamp(grad_output, min=0.0)
        return grad_input


# Replace ReLU with GuidedBackpropReLU
def replace_relu_with_guided(model):
    for name, module in model.named_children():
        if isinstance(module, nn.ReLU):
            setattr(model, name, GuidedBackpropReLU())
        else:
            replace_relu_with_guided(module)


# Preprocess the input image
def preprocess_image(image_path):
    preprocess = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])
    image = Image.open(image_path).convert('RGB')
    return preprocess(image).unsqueeze(0)  # Add batch dimension


# Guided Backpropagation function
def guided_backpropagation(model, input_image, target_class):
    model.eval()  # Set model to evaluation mode
    input_image.requires_grad = True  # Enable gradient computation for input

    replace_relu_with_guided(model)  # Replace all ReLU activations

    output = model(input_image)  # Forward pass
    loss = output[0, target_class]  # Target class loss
    model.zero_grad()  # Zero out gradients
    loss.backward()  # Backward pass

    return input_image.grad.data  # Return gradients w.r.t. input image


# Visualize the gradients
def visualize_gradients(gradients):
    gradients = gradients[0].cpu().numpy()  # Remove batch dimension
    gradients = np.moveaxis(gradients, 0, -1)  # Rearrange channels to last dimension
    gradients = (gradients - gradients.min()) / (gradients.max() - gradients.min())  # Normalize
    plt.imshow(gradients)
    plt.axis('off')
    plt.show()


# Main script
if __name__ == "__main__":
    # Load a pretrained model
    model = models.vgg16(pretrained=True)

    # Load and preprocess the input image
    image_path = "/content/g2.jpeg"  # Replace with the path to your image
    input_image = preprocess_image(image_path)

    # Define target class (e.g., 'bull mastiff' class in ImageNet)
    target_class = 243

    # Move to device (CPU/GPU)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = model.to(device)
    input_image = input_image.to(device)

    # Perform Guided Backpropagation
    gradients = guided_backpropagation(model, input_image, target_class)

    # Visualize the gradients
    visualize_gradients(gradients)


In [ ]:
gradients = (gradients - gradients.min()) / (gradients.max() - gradients.min())  # Normalize


In [ ]:
print("Gradient shape:", gradients.shape)


In [ ]:
output = model(input_image)
print("Model output:", output)


In [ ]:
gradients = guided_backpropagation(model, input_image, target_class)
print("Gradients shape:", gradients.shape)  # Should be [1, 3, 224, 224] for a single image


In [ ]:
plt.imshow(gradients[0].cpu().numpy()[0], cmap='hot')  # Display the first channel's gradients
plt.show()